In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from datetime import datetime

from collections import Counter

import pandas as pd
import time
import sys, os
import re
import json
import random
import numpy as np



def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def find_tagName(tag_name, browser):
    return browser.find_element(By.TAG_NAME, tag_name)

def finds_tagName(tag_name, browser):
    return browser.find_elements(By.TAG_NAME, tag_name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

In [2]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")
    
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [3]:
def get_url(browser ,url):
    browser.get(url)
    time.sleep(1)

In [4]:
dd1 = pd.read_excel("표본_기업명.xlsx")
dd = pd.read_excel("C:\\Users\\yms06\\OneDrive\\바탕 화면\\설비투자 상위층, 전수층 최종본 (1).xlsx")

In [5]:
ddf = dd[dd['기업명'].isin(dd1['기업명'])]
url_list2 = ddf['웹입력URL'].to_list()

In [19]:
ddf = ddf.drop_duplicates(subset='기업명')
ddf.shape

(988, 47)

In [20]:
company_list = ddf['기업명'].to_list()

In [23]:
ddf.to_excel("표본_입력.xlsx", index=False)

In [4]:
dd= pd.read_excel("표본_입력.xlsx")
dd.shape

(988, 49)

In [5]:
company_size_list = []

for c in dd['기업규모']:
    if c in "대기업":
        company_size_list.append(1)
    elif c in "중견기업":
        company_size_list.append(2)
    elif c in "중소기업":
        company_size_list.append(3)
    else:
        company_size_list.append(3)

In [17]:
url_list = dd['웹입력URL'].to_list()
gender_list = dd['성별'].to_list()
region_list = dd['본주소'].to_list()
company_size_list = company_size_list
company_name_list = dd['기업명']

In [37]:
browser = open_browser()
get_url(browser, url_list[1])

In [13]:
count = 1
browser = open_browser()

current_urls = []
f_urls = []
company_names = []
error_urls = []

for url, gender, address, company_size in zip(url_list, gender_list, region_list, company_size_list):
    try:
        current_url = browser.current_url
        if 'svyI001' in current_url:
            get_url(browser, url)
            time.sleep(1)
            first_field_page(browser, gender, address, company_size)
            time.sleep(1)
            first_one_page_field(browser)
            time.sleep(1)
        elif 'svyI003' in current_url:
    except:
        error_urls.append(url)
        pass

In [21]:
count = 1
browser = open_browser()

current_urls = []
f_urls = []
company_names = []
error_urls = []

field_list = [[],[],[],[],[]]
field1 = []
field2 = []
field3 = []
field4 = []
field5 = []

for url, gender, address, company_size, company_name in zip(url_list, gender_list, region_list, company_size_list, company_name_list):
    try:
        get_url(browser, url)
        current_url = browser.current_url
        time.sleep(1)
        if 'svyI001' in current_url:
            get_url(browser, url)
            time.sleep(1)
            first_field_page(browser, gender, address, company_size)
            time.sleep(1)
            first_one_page_field(browser)
            time.sleep(1)
            ### 토지 및 토지개발
            field_list[0].append(find_name('A_1_2', browser).get_attribute('value'))
            ### 건물 및 구축물
            field_list[1].append(find_name('A_1_3', browser).get_attribute('value'))
            ### 기계장치
            field_list[2].append(find_name('A_1_4', browser).get_attribute('value'))
            ### 운수장비(선박, 차량)
            field_list[3].append(find_name('A_1_5', browser).get_attribute('value'))
            ### 기타
            field_list[4].append(find_name('A_1_6', browser).get_attribute('value'))
            f_urls.append(url)
            company_names.append(company_name)
            time.sleep(1)
        
        elif 'svyI003' in current_url:
            ### 토지 및 토지개발
            field_list[0].append(find_name('A_1_2', browser).get_attribute('value'))
            ### 건물 및 구축물
            field_list[1].append(find_name('A_1_3', browser).get_attribute('value'))
            ### 기계장치
            field_list[2].append(find_name('A_1_4', browser).get_attribute('value'))
            ### 운수장비(선박, 차량)
            field_list[3].append(find_name('A_1_5', browser).get_attribute('value'))
            ### 기타
            field_list[4].append(find_name('A_1_6', browser).get_attribute('value'))
            f_urls.append(url)
            company_names.append(company_name)
            time.sleep(1)
            print(count)
            count += 1
    except:
        error_urls.append(url)
        print(count)
        count += 1
        pass
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
for i in error_urls:
    try:
        get_url(browser, i)
        time.sleep(1)
    except:
        pass

In [32]:
df22 = pd.DataFrame({
    'company_name':company_names, 
    'url' : f_urls,
    '토지 및 토지개발' : field_list[0],
    '건물 및 구축물' : field_list[1],
    '기계장치' : field_list[2],
    '운수장비(선박, 차량)' : field_list[3],
    '기타' : field_list[4]
}) 

df22.shape

(970, 7)

In [34]:
df22.to_excel("표본_설비값_추출.xlsx")

In [45]:
def countLetters(word):
    counter = {}
    for letter in word:
        if letter not in counter:
            counter[letter] = 0
        counter[letter] += 1
    return counter

countLetters(final_current_urls)

{'svyI001': 2158, 'svyI003': 54, 'svyI002': 6, 'svyI006': 1}

In [7]:
def first_field_page(browser, gender, address, company_size):
    field_data = find_id('objS5_2', browser).get_attribute('value')
    
    if field_data == '':
        # 성별 click
        if gender:
            find_id('objS2a_1', browser).click()
        else:
            find_id('objS2a_2', browser).click()

        time.sleep(.5)

        # region input
        find_id('btnZipSearchS5', browser).click()
        time.sleep(1)

        try:
            iframe_1 = find_xpath('/html/body/div/form/table[1]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/table[4]/tbody/tr/td/center/table/tbody/tr[6]/td/table[2]/tbody/tr/td/div/div/iframe', browser)
            browser.switch_to.frame(iframe_1)
            time.sleep(.5)
            browser.switch_to.frame('__daum__viewerFrame_1')
        except:
            browser.switch_to.frame('__daum__viewerFrame_1')

        time.sleep(.5)

        region_name_input = find_id('region_name', browser)
        region_name_input.send_keys(address)
        time.sleep(.5)
        region_name_input.send_keys('\n')

        time.sleep(2)

        region_click = find_xpath('/html/body/div[1]/div/div[2]/ul/li/dl/dd[1]/span/button', browser)
        region_click.click()

        browser.switch_to.default_content()

        time.sleep(1)

        ## Enter detailed address
        find_id('objS5_2', browser).send_keys('.')
        time.sleep(.5)

        # Click Enterprise(Company) Size
        if company_size == 1:
            find_id('objS6_1', browser).click()
        elif company_size == 2:
            find_id('objS6_2', browser).click()
        else: find_id('objS6_3', browser).click()

        time.sleep(1)

        # Click Export Ratio
        if company_size == 2 or company_size == 3:
            find_id('objS7_1', browser).click()
        elif company_size == 1:
            find_id('objS7_2', browser).click()
#         elif company_size == 3:
#             find_id('objS7_3', browser).click()
#         else: find_id('objS7_4', browser).click()
    
    time.sleep(1.5)
    find_id('BtnNextPage', browser).click()

In [8]:
def first_one_page_field(browser):
    time.sleep(1)
    find_id('BtnNextPage', browser).click()

In [9]:
def second_field_page(
    browser, f1, f2, f3, f4, f5, f6
    ):
    # 2022년 (실적)
    ## 자산형태별 설비투자 (단위 : 억원)
    ### 토지 및 토지개발
    find_name('A_1_2', browser).send_keys()
    ### 건물 및 구축물
    find_name('A_1_3', browser).send_keys()
    ### 기계장치
    find_name('A_1_4', browser).send_keys()
    ### 운수장비(선박, 차량)
    find_name('A_1_5', browser).send_keys()
    ### 기타
    find_name('A_1_6', browser).send_keys()
    
    ## 설비자금조달방법 (단위 : 억원)
    ### 주식
    find_name('B_1_3', browser).send_keys()
    ### 회사채
    find_name('B_1_4', browser).send_keys()
    ### 은행
    find_name('B_1_6', browser).send_keys()
    ### 비은행
    find_name('B_1_7', browser).send_keys()
    ### 기타
    find_name('B_1_8', browser).send_keys()
    ### 내부자금
    find_name('B_1_9', browser).send_keys()
    
    ## 투자동기별 설비투자 (단위 : 억원)
    ### (유형별) 신제품 생산
    find_name('C_1_3', browser).send_keys()
    ### (유형별) 설비의 확장
    find_name('C_1_4', browser).send_keys()
    ### (수요별) 국내수요대비
    find_name('C_1_5', browser).send_keys()
    ### (수요별) 수출수요대비
    find_name('C_1_6', browser).send_keys()
    ### 설비의 유지보수
    find_name('C_1_7', browser).send_keys()
    ### 자동화
    find_name('C_1_8', browser).send_keys()
    ### 에너지 및 환경관련 투자
    find_name('C_1_9', browser).send_keys()
    ### 연구개발 투자
    find_name('C_1_10', browser).send_keys()
    ### 기타
    find_name('C_1_11', browser).send_keys()
    
    
    ## 연구개발 투자 (단위 : 억원)
    ### 기계장치(유형자산취득)
    find_name('C1_1_2', browser).send_keys()
    ### 토지 및 건물 (유형자산취득)
    find_name('C1_1_3', browser).send_keys()
    
    #-------------------------------------------------
    
    # 2023년 (잠정)
    ## 자산형태별 설비투자 (단위 : 억원)
    ### 토지 및 토지개발
    find_name('A_2_2', browser).send_keys()
    ### 건물 및 구축물
    find_name('A_2_3', browser).send_keys()
    ### 기계장치
    find_name('A_2_4', browser).send_keys()
    ### 운수장비(선박, 차량)
    find_name('A_2_5', browser).send_keys()
    ### 기타
    find_name('A_2_6', browser).send_keys()
    
    ## 설비자금조달방법 (단위 : 억원)
    ### 주식
    find_name('B_2_3', browser).send_keys()
    ### 회사채
    find_name('B_2_4', browser).send_keys()
    ### 은행
    find_name('B_2_6', browser).send_keys()
    ### 비은행
    find_name('B_2_7', browser).send_keys()
    ### 기타
    find_name('B_2_8', browser).send_keys()
    ### 내부자금
    find_name('B_2_9', browser).send_keys()
    
    ## 투자동기별 설비투자 (단위 : 억원)
    ### (유형별) 신제품 생산
    find_name('C_2_3', browser).send_keys()
    ### (유형별) 설비의 확장
    find_name('C_2_4', browser).send_keys()
    ### (수요별) 국내수요대비
    find_name('C_2_5', browser).send_keys()
    ### (수요별) 수출수요대비
    find_name('C_2_6', browser).send_keys()
    ### 설비의 유지보수
    find_name('C_2_7', browser).send_keys()
    ### 자동화
    find_name('C_2_8', browser).send_keys()
    ### 에너지 및 환경관련 투자
    find_name('C_2_9', browser).send_keys()
    ### 연구개발 투자
    find_name('C_2_10', browser).send_keys()
    ### 기타
    find_name('C_2_11', browser).send_keys()
    
    
    ## 연구개발 투자 (단위 : 억원)
    ### 기계장치(유형자산취득)
    find_name('C1_2_2', browser).send_keys()
    ### 토지 및 건물 (유형자산취득)
    find_name('C1_2_3', browser).send_keys()
    
    #-------------------------------------------------
    
    # 2024년 (계획)
    ## 자산형태별 설비투자 (단위 : 억원)
    ### 토지 및 토지개발
    find_name('A_3_2', browser).send_keys()
    ### 건물 및 구축물
    find_name('A_3_3', browser).send_keys()
    ### 기계장치
    find_name('A_3_4', browser).send_keys()
    ### 운수장비(선박, 차량)
    find_name('A_3_5', browser).send_keys()
    ### 기타
    find_name('A_3_6', browser).send_keys()
    
    ## 설비자금조달방법 (단위 : 억원)
    ### 주식
    find_name('B_3_3', browser).send_keys()
    ### 회사채
    find_name('B_3_4', browser).send_keys()
    ### 은행
    find_name('B_3_6', browser).send_keys()
    ### 비은행
    find_name('B_3_7', browser).send_keys()
    ### 기타
    find_name('B_3_8', browser).send_keys()
    ### 내부자금
    find_name('B_3_9', browser).send_keys()
    
    ## 투자동기별 설비투자 (단위 : 억원)
    ### (유형별) 신제품 생산
    find_name('C_3_3', browser).send_keys()
    ### (유형별) 설비의 확장
    find_name('C_3_4', browser).send_keys()
    ### (수요별) 국내수요대비
    find_name('C_3_5', browser).send_keys()
    ### (수요별) 수출수요대비
    find_name('C_3_6', browser).send_keys()
    ### 설비의 유지보수
    find_name('C_3_7', browser).send_keys()
    ### 자동화
    find_name('C_3_8', browser).send_keys()
    ### 에너지 및 환경관련 투자
    find_name('C_3_9', browser).send_keys()
    ### 연구개발 투자
    find_name('C_3_10', browser).send_keys()
    ### 기타
    find_name('C_3_11', browser).send_keys()
    
    
    ## 연구개발 투자 (단위 : 억원)
    ### 기계장치(유형자산취득)
    find_name('C1_3_2', browser).send_keys()
    ### 토지 및 건물 (유형자산취득)
    find_name('C1_3_3', browser).send_keys()

In [10]:
def third_page_field(browser, ):
    field1 = find_id('objE_2', browser).is_selected()
    field2 = find_id('objE_3', browser).is_selected()
    field3 = find_id('objE_4', browser).is_selected()
    field4 = find_id('objE_5', browser).is_selected()
    field5 = find_id('objE_6', browser).is_selected()
    field6 = find_id('objE_7', browser).is_selected()
    field7 = find_id('objE_8', browser).is_selected()
    field8 = find_id('objE_9', browser).is_selected()
    field9 = find_id('objE_10', browser).is_selected()

    field_list = [field1, field2, field3, field4, field5, field6, field7, field8, field9]
    
    ic1 = find_id('objE_2', browser)
    ic2 = find_id('objE_3', browser)
    ic3 = find_id('objE_4', browser)
    ic4 = find_id('objE_5', browser)
    ic5 = find_id('objE_6', browser)
    ic6 = find_id('objE_7', browser)
    ic7 = find_id('objE_8', browser)
    ic8 = find_id('objE_9', browser)
    ic9 = find_id('objE_10', browser)
    
    ic_list = [ic1, ic2, ic3, ic4, ic5, ic6, ic7, ic8, ic9]

    if not any(field_list):
        pass
        
        ## 기타
        # find_id('objE_11', browser).click()
        # find_id('objE_11_etc', browser).send_keys('test')

In [11]:
def fourth_page_field(browser):
    field1 = find_id('objF_1', browser).is_selected()
    field2 = find_id('objF_2', browser).is_selected()
    field3 = find_id('objF_3', browser).is_selected()

    field_list = [field1, field2, field3]
    
    randint = random.randint(0, 2)
    
    # 투자에 대한 세액 공제 확대
    ic1 = find_id('objF_1', browser)
    # 저리의 정책금융 제공
    ic2 = find_id('objF_2', browser)
    # 투자관련 규제완화
    ic3 = find_id('objF_3', browser)

    ic_list = [ic1, ic2, ic3]

    if not any(field_list):
        ic_list[randint].click()
        
    time.sleep(1)
    find_id('BtnNextPage', browser).click()

In [12]:
def fiveth_page_field(browser, freebies_status1, freebies_status2):
    field1 = find_id('objDQ0a_1', browser).is_selected()
    field2 = find_id('objDQ0a_2', browser).is_selected()
    field3 = find_id('objDQ0b_1', browser).is_selected()
    field4 = find_id('objDQ0b_2', browser).is_selected()

    field_list = [field1,field2, field3, field4]
    
    if not any(field_list):
        # 답례품 수령 희망 여부
        if freebies_status1:
            ## 예
            find_id('objDQ0a_1', browser).click()
        else:
            ## 아니요
            find_id('objDQ0a_2', browser).click()

        # 설비투자계획조사 결과(책자) 수령 희망여부
        if freebies_status2:
            ## 예
            find_id('objDQ0b_1', browser).click()
        else:
            ## 아니요
            find_id('objDQ0b_2', browser).click()
            
    time.sleep(1)
    find_id('BtnNextPage', browser).click()

In [11]:
dm = '개발팀'
ps = '사원'
name = '아무개'
tel = '02-03030-0303'
email = 'yyyyy1234@naver.com'
address = '서울시 광진구 영화사로11길 6'
sub_address = '101호'

In [12]:
def sixth_page_field(browser, dm, ps, name, tel, email, address, sub_address):
    field1 = find_id('objDQ1', browser).get_attribute('value')
    field2 = find_id('objDQ2', browser).get_attribute('value')
    field3 = find_id('objDQ3', browser).get_attribute('value')
    field4 = find_id('objDQ4', browser).get_attribute('value')
    field5 = find_id('objDQ5', browser).get_attribute('value')

    field_list = [field1, field2, field3, field4, field5]

    if not any(field_list):
        find_id('objDQ1', browser).send_keys(dm)
        time.sleep(.5)
        find_id('objDQ2', browser).send_keys(ps)
        time.sleep(.5)
        find_id('objDQ3', browser).send_keys(name)
        time.sleep(.5)
        find_id('objDQ4', browser).send_keys(tel)
        time.sleep(.5)
        find_id('objDQ5', browser).send_keys(email)

        time.sleep(1)

        find_id('btnZipSearchDQ6', browser).click()
        time.sleep(1)

        try:
            iframe_1 = find_xpath('/html/body/div/form/table[1]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/table[3]/tbody/tr/td/center/table/tbody/tr[6]/td/table[2]/tbody/tr/td/div/div/iframe', browser)
            browser.switch_to.frame(iframe_1)
            time.sleep(.5)
            iframe_2 = find_xpath('/html/body/iframe', browser)
            browser.switch_to.frame(iframe_2)
        #     browser.switch_to.frame('__daum__viewerFrame_2')
        except:
            iframe_2 = find_xpath('/html/body/iframe', browser)
            browser.switch_to.frame(iframe_2)
        #     browser.switch_to.frame('__daum__viewerFrame_3')

        time.sleep(.5)

        region_name_input = find_id('region_name', browser)
        region_name_input.send_keys(address)
        time.sleep(.5)
        region_name_input.send_keys('\n')

        time.sleep(1.5)

        region_click = find_xpath('/html/body/div[1]/div/div[2]/ul/li/dl/dd[1]/span/button', browser)
        region_click.click()

        browser.switch_to.default_content()

        time.sleep(.5)

        find_id('objDQ6_2', browser).send_keys(sub_address)
        
    time.sleep(1)
    find_id('BtnNextPage', browser).click()

In [67]:
browser = open_browser()
get_url(browser ,url)

In [68]:
browser = open_browser()

url_list_3 = []
url_list_6 = []

for url in url_list:
    try:
        get_url(browser ,url)
    #     time.sleep(1)
        current_url = browser.current_url

        if 'svyI001' in current_url:
            first_field_page(browser, gender, address, sub_address, company_size, export_ratio)
            time.sleep(1)
            first_one_page_field(browser)
            time.sleep(1)
            second_field_page(browser, f1, f2, f3, f4, f5, f6)
            time.sleep(1)
            third_page_field(browser, )
            time.sleep(1)
            fourth_page_field(browser)
            time.sleep(1)
            fiveth_page_field(browser, freebies_status1, freebies_status2)
            time.sleep(1)
            sixth_page_field(browser, dm, ps, name, tel, email, address, sub_address)
        elif 'svyI002' in current_url:
            first_one_page_field(browser)
            time.sleep(1)
            second_field_page(browser, f1, f2, f3, f4, f5, f6)
            time.sleep(1)
            third_page_field(browser, )
            time.sleep(1)
            fourth_page_field(browser)
            time.sleep(1)
            fiveth_page_field(browser, freebies_status1, freebies_status2)
            time.sleep(1)
            sixth_page_field(browser, dm, ps, name, tel, email, address, sub_address)
        elif 'svyI003' in current_url:
            second_field_page(browser, f1, f2, f3, f4, f5, f6)
            time.sleep(1)
            third_page_field(browser, )
            time.sleep(1)
            fourth_page_field(browser)
            time.sleep(1)
            fiveth_page_field(browser, freebies_status1, freebies_status2)
            time.sleep(1)
            sixth_page_field(browser, dm, ps, name, tel, email, address, sub_address)
            url_list_3.append(url)
        elif 'svyl005' in current_url:
            third_page_field(browser, )
            time.sleep(1)
            fourth_page_field(browser)
            time.sleep(1)
            fiveth_page_field(browser, freebies_status1, freebies_status2)
            time.sleep(1)
            sixth_page_field(browser, dm, ps, name, tel, email, address, sub_address)
        elif 'svyI006' in current_url:
            fourth_page_field(browser)
            time.sleep(1)
            fiveth_page_field(browser, freebies_status1, freebies_status2)
            time.sleep(1)
            sixth_page_field(browser, dm, ps, name, tel, email, address, sub_address)
    except:
        pass

svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI003 들어옴3
svyI003 들어옴3
svyI001 들어옴1
svyI001 들어옴1
svyI003 들어옴3
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI003 들어옴3
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1

svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI003 들어옴3
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI003 들어옴3
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1

svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI003 들어옴3
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI003 들어옴3
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1

svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI002 들어옴2
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1
svyI001 들어옴1

In [70]:
url_list_6

['https://rnr.surveybox.kr/?pid=S16269h8ufyh&grpid=list&resid=14510&fwid=S16269h8ufyhz56yr']